In [ ]:
import pathlib as pl
import datetime as dt

import shapely
import pandas as pd
import r5py as r5
import r5py.sampledata.helsinki as hs
import pyrosm as pr
import geopandas as gpd
from loguru import logger

DATA_DIR = pl.Path("../data") 

# Setup

In [ ]:
WGS84 = "epsg:4326"
NZTM = "epsg:2193"

%ls {DATA_DIR}

# Load Auckland OSM and GTFS DATA
%time akl_pbf_path = pr.get_data("Auckland", directory=DATA_DIR)
akl_gtfs_path = DATA_DIR / "auckland_gtfs_20230824.zip"



In [ ]:
points = gpd.read_file(DATA_DIR / "auckland_points.geojson").assign(id=lambda x: x.index)
display(points)
display(points.explore())
# %time transport_network = r5.TransportNetwork(akl_pbf_path, [akl_gtfs_path])
# transport_modes = [
#     r5.TransportMode.TRANSIT,
#     r5.TransportMode.WALK,
# ]


# Write an isochrone function for r5py

To address [this Github issue](https://github.com/r5py/r5py/issues/311) .
Start by porting the [r5r isochrone code](https://github.com/ipeaGIT/r5r/blob/master/r-package/R/isochrone.R) to Python.

In [ ]:
def get_osm_nodes(transport_network) -> gpd.GeoDataFrame:
    import com.conveyal.r5
    
    k = com.conveyal.r5.streets.VertexStore.FIXED_FACTOR
    v = transport_network._transport_network.streetLayer.vertexStore
    lonlats = zip(list(v.fixedLons.toArray()), list(v.fixedLats.toArray()))
    nodes = gpd.GeoDataFrame(
        geometry=[shapely.Point(lon / k, lat / k) for lon, lat in lonlats],
        crs="epsg:4326",
    )
    nodes["id"] = nodes.index
    return nodes

def get_osm_nodes_and_edges(osm_pbf_path: pl.Path, network_type: str="all") -> gpd.GeoDataFrame:
    """
    Read the OSM protobuf file at the given path and extract from it nodes
    of the given network type.
    Return the nodes as a GeoDataFrame with the columns 'id' (OSM ID), 'geometry'.
    Uses Pyrosm and can be slow at around 40 seconds for Helsinki.
    """
    osm = pr.OSM(str(osm_pbf_path))
    return osm.get_network(network_type="all", nodes=True)


In [ ]:
def isochrone_ch(
    transport_network: r5.TransportNetwork,
    transport_modes: list[r5.TransportMode],
    origins: gpd.GeoDataFrame,
    time_bounds: list[float],
    departure: dt.datetime|None=None,
    snap_to_network: bool|int=False,
    sample_frac: float=0.8,
    concave_hull_ratio=0.15,
    **kwargs: dict,
) -> gpd.GeoDataFrame:
    """
    Return a GeoDataFrame of isochrones (polygons) from the given origins and 
    of the given time bounds in minutes.
    Use the given transport nework, transport modes, and departure datetime, 
    the latter of which defaults to the current datetime.
       
    Further customise the isochrone calculation as follows.
    
    - Use a random sample of ``sample_frac`` of all the underlying OSM nodes as potential destinations.
    - When making the isochrones using concave hulls of reachable points, use the given concave hull ratio.
    - Snap the origin points to the street network before routing if and only if ``snap_to_network``
      If ``True``, the default search radius 
      (defined in com.conveyal.r5.streets.StreetLayer.LINK_RADIUS_METERS) is used; 
      if int, then use that many meters as the search radius for snapping.
    - Pass in any keyword arguments accepted by :class:`r5py.RegionalTask`, 
      e.g. `departure_time_window`, `percentiles`, `max_time_walking`.
    
    """
    time_bounds = sorted(set(time_bounds))
    
    # Use a random sample of network nodes as destination points
    logger.info("Get OSM nodes to route to")
    osm_nodes = get_osm_nodes(transport_network).sample(frac=sample_frac, random_state=1)

    # Compute travel times
    logger.info("Compute travel times")
    ttm = r5.TravelTimeMatrixComputer(
        transport_network,
        origins=origins,
        destinations=osm_nodes,
        departure=departure,
        transport_modes=transport_modes,
        snap_to_network=snap_to_network,
        max_time=dt.timedelta(seconds=(time_bounds[-1] + 5)* 60),  # Prune search tree
        **kwargs,
    )
    f = ttm.compute_travel_times().dropna()
    if f.empty:
        return gpd.GeoDataFrame()

    # Build isochrones as concave hulls of reachable points
    logger.info("Build isochrones")
    records = []
    for from_id, group in f.groupby("from_id"):
        for time_bound in time_bounds:
            reachable_nodes = osm_nodes.merge(
                group
                .loc[lambda x: x["travel_time"] <= time_bound]
                .rename(columns={"to_id": "id"})
            )
            iso = shapely.concave_hull(reachable_nodes.unary_union, ratio=concave_hull_ratio)
            records.append(
                {
                    "origin_id": from_id,
                    "time_bound": time_bound,
                    "geometry": iso,
                }
            )

    logger.info("Build GeoDataFrame")
    return gpd.GeoDataFrame(pd.DataFrame.from_records(records), crs=WGS84)
  

def isochrone_be(
    transport_network: r5.TransportNetwork,
    transport_modes: list[r5.TransportMode],
    osm_pbf_path: pl.Path,
    origins: gpd.GeoDataFrame,
    time_bounds: list[float],
    meters_crs:str,
    buffer:float=10,
    simplify:float=0,
    osm_nodes: gpd.GeoDataFrame|None=None,
    osm_edges: gpd.GeoDataFrame|None=None,
    departure: dt.datetime|None=None,
    snap_to_network:bool|int=False,
    **kwargs: dict,
) -> gpd.GeoDataFrame:
    """
    Return a GeoDataFrame of isochrones (polygons) from the given origins and 
    of the given time bounds in minutes.
    Use the given transport nework, transport modes, the path to OSM protobuf file
    underlying the transport network, and departure datetime, 
    the latter of which defaults to the current datetime.

    Further customise the isochrone calculation as follows.
    
    - Use a random sample of ``sample_frac` `of all transport network nodes as potential destinations.
    - Snap the origin points to the street network before routing if and only if ``snap_to_network``
      If ``True``, the default search radius 
      (defined in com.conveyal.r5.streets.StreetLayer.LINK_RADIUS_METERS) is used; 
      if int, then use that many meters as the search radius for snapping.
    - Pass in any keyword arguments accepted by :class:`r5py.RegionalTask`, 
      e.g. `departure_time_window`, `percentiles`, `max_time_walking`.
    
    """
    time_bounds = sorted(set(time_bounds))
    
    # Use a random sample of network nodes as destination points
    logger.info("Get OSM nodes and edges")
    if osm_nodes is None or osm_edges is None:
        osm_nodes, osm_edges = get_osm_nodes_and_edges(osm_pbf_path)

    osm_nodes = osm_nodes.filter(["id", "geometry"])
    osm_edges = (
        osm_edges
        .filter(["v", "geometry"])
        .rename(columns={"v": "id"})
    )

    # Compute travel times
    logger.info("Compute travel times")
    ttm = r5.TravelTimeMatrixComputer(
        transport_network,
        origins=origins,
        destinations=osm_nodes,
        departure=departure,
        transport_modes=transport_modes,
        snap_to_network=snap_to_network,
        max_time=dt.timedelta(seconds=time_bounds[-1] * 60),
        **kwargs,
    )
    bins = [0] + time_bounds
    labels = time_bounds
    f = ttm.compute_travel_times().dropna()

    if f.empty:
        return gpd.GeoDataFrame()

    # Build isochrones as buffered edges
    logger.info("Build isochrones")
    records = []
    for from_id, group in f.groupby("from_id"):
        for time_bound in time_bounds:
            reachable_nodes = osm_nodes.merge(
                group
                .loc[lambda x: x["travel_time_bin"] <= time_bound]
                .rename(columns={"to_id": "id"})
            )
            edge_blob = (
                osm_edges
                .merge(reachable_nodes.filter(["id"]))
                .to_crs(meters_crs)
                .unary_union
            )
            records.append(
                {
                    "origin_id": from_id,
                    "time_bound": time_bound,
                    "geometry": edge_blob,
                }
            )

    logger.info("Build GeoDataFrame")
    return (
        gpd.GeoDataFrame(pd.DataFrame.from_records(records), crs=meters_crs)
        .assign(geometry=lambda x: x.buffer(buffer).simplify(simplify))
        .to_crs("epsg:4326")
    )
  

In [ ]:
%%time

isos_ch = isochrone_ch(
    transport_network=transport_network, 
    transport_modes=transport_modes,
    origins=points.iloc[:1],
    departure=dt.datetime(2023, 8, 28, 8, 0, 0),
    time_bounds=[15, 30, 45],
    departure_time_window=dt.timedelta(seconds=30*60),
    concave_hull_ratio=0.15,
)
display(isos_ch)

display(
    isos_ch
    .loc[lambda x: x["time_bound"] == 45]
    .explore()
)